In [4]:
import pyreadstat as prs
from scipy import stats
import pandas as pd
import numpy as np
df, meta= prs.read_sav("D:/myData/E0801.sav", encoding='gb2312') #读入数据，df为dataframe，meta为附加信息
df

,datas,groups
0,0.81,1.0
1,0.92,1.0
2,0.82,1.0
3,0.87,1.0
4,0.91,1.0
5,0.83,1.0
6,0.86,1.0
7,0.92,1.0
8,0.90,1.0
9,0.50,2.0


In [8]:
import statsmodels.sandbox.stats.multicomp
from statsmodels.sandbox.stats.multicomp import tukeyhsd
from statsmodels.compat.python import lzip
from statsmodels.iolib.table import SimpleTable
from statsmodels.sandbox.stats.multicomp import TukeyHSDResults
from statsmodels.sandbox.stats.multicomp import GroupsStats
from statsmodels.sandbox.stats.multicomp import varcorrection_pairs_unbalanced
from statsmodels.sandbox.stats.multicomp import get_tukeyQcrit2
from statsmodels.sandbox.stats.multicomp import get_tukey_pvalue
class myMultiComparison(object):

    def __init__(self, data, groups, group_order=None):

        if len(data) != len(groups):
            raise ValueError('data has %d elements and groups has %d' % (len(data), len(groups)))
        self.data = np.asarray(data)
        self.groups = groups = np.asarray(groups)

        # Allow for user-provided sorting of groups
        if group_order is None:
            self.groupsunique, self.groupintlab = np.unique(groups,
                                                            return_inverse=True)
        else:
            #check if group_order has any names not in groups
            for grp in group_order:
                if grp not in groups:
                    raise ValueError(
                            "group_order value '%s' not found in groups" % grp)
            self.groupsunique = np.array(group_order)
            self.groupintlab = np.empty(len(data), int)
            self.groupintlab.fill(-999)  # instead of a nan
            count = 0
            for name in self.groupsunique:
                idx = np.where(self.groups == name)[0]
                count += len(idx)
                self.groupintlab[idx] = np.where(self.groupsunique == name)[0]
            if count != self.data.shape[0]:
                #raise ValueError('group_order does not contain all groups')
                # warn and keep only observations with label in group_order
                import warnings
                warnings.warn('group_order does not contain all groups:' +
                              ' dropping observations', ValueWarning)

                mask_keep = self.groupintlab != -999
                self.groupintlab = self.groupintlab[mask_keep]
                self.data = self.data[mask_keep]
                self.groups = self.groups[mask_keep]

        if len(self.groupsunique) < 2:
            raise ValueError('2 or more groups required for multiple comparisons')

        self.datali = [self.data[self.groups == k] for k in self.groupsunique] #根据组别分割data
        self.pairindices = np.triu_indices(len(self.groupsunique), 1)  #将要比较的两组的索引分成两个tuple
        self.nobs = self.data.shape[0] #总例数
        self.ngroups = len(self.groupsunique) #组数
    
    def snk_q(self,alpha=0.05):
        """
        imitate tukeyhsd()
        MSerror: mean squares within group or mean squares within error
         
        """
        dof = self.nobs - self.ngroups #（误差）组内自由度
        array_index, array_counts = np.unique(self.groupintlab, return_counts=True) #获取每组索引以及例数,重点是每组的例数
        SSerror = sum(np.asarray(pd.DataFrame(self.datali).var(axis=1, ddof=0)) * array_counts) #计算每组的离均差平方和（通过方差*每组例数）
        MSerror = SSerror / dof  #组内（误差）均方
        
        groupstats = GroupsStats(np.column_stack([self.data, self.groupintlab]),useranks=False)
        gmeans = groupstats.groupmean #各组均值，arrary
        meandiffs_ = gmeans - gmeans[:,None] #求出两不同维度矩阵的差，返回二维矩阵
        meandiffs = meandiffs_[self.pairindices[0], self.pairindices[1]] #以上两行代码返回上三角的数据
        meandiffs = np.abs(meandiffs) #取绝对值,得到均差绝对值
        
        array_counts_reciprocal = 1 / array_counts #求倒数
        array_counts_reciprocal = array_counts_reciprocal + array_counts_reciprocal[:,None]
        array_counts_reciprocal = array_counts_reciprocal[self.pairindices[0], self.pairindices[1]]#方法如求均差一样，求出（1/n1 + 1/n2)

        qvalue = meandiffs / np.sqrt(MSerror / 2 * array_counts_reciprocal) #求出q值
        steps = np.abs(np.asarray(self.pairindices[0])-np.asarray(self.pairindices[1]))+1 #步长
        
        #以下代码制SimpleTable表
        resarr = np.array(lzip(self.pairindices[0]+1, self.pairindices[1]+1, np.round(meandiffs,4), steps, np.round(qvalue,4)),
                          dtype=[('group1', object),('group2', object),('meandiffs', float),('steps', float),('qvalue', float)])
        results_table = SimpleTable(resarr, headers=resarr.dtype.names)
        results_table.title = 'Multiple Comparison of Means - SNK, FWER=%4.2f' % alpha
        return results_table
        
#类结束

In [9]:
a = myMultiComparison(df['datas'],groups=df['groups'])
a.snk_q()

group1,group2,meandiffs,steps,qvalue
1,2,0.3689,2.0,29.4241
1,3,0.5144,3.0,41.0342
1,4,0.6256,4.0,49.8968
2,3,0.1456,2.0,11.6101
2,4,0.2567,3.0,20.4728
3,4,0.1111,2.0,8.8627
